In [75]:
!pip3 install catboost
!pip install hyperopt

In [76]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import hyperopt
from numpy.random import RandomState
from catboost.utils import select_threshold

In [77]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [78]:
csv_path = "drive/MyDrive/trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)

In [79]:
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
print(x_columns)
print(y_columns)

['trade_volume-BUY-6-sec', 'moving_average_price-BUY-6-sec', 'weighted_moving_average_price-BUY-6-sec', 'exp_moving_average_price-BUY-6-sec', 'stochastic_oscillator-BUY-6-sec', 'trade_volume-BUY-12-sec', 'moving_average_price-BUY-12-sec', 'weighted_moving_average_price-BUY-12-sec', 'exp_moving_average_price-BUY-12-sec', 'stochastic_oscillator-BUY-12-sec', 'trade_volume-BUY-18-sec', 'moving_average_price-BUY-18-sec', 'weighted_moving_average_price-BUY-18-sec', 'exp_moving_average_price-BUY-18-sec', 'stochastic_oscillator-BUY-18-sec', 'trade_volume-BUY-24-sec', 'moving_average_price-BUY-24-sec', 'weighted_moving_average_price-BUY-24-sec', 'exp_moving_average_price-BUY-24-sec', 'stochastic_oscillator-BUY-24-sec', 'trade_volume-BUY-30-sec', 'moving_average_price-BUY-30-sec', 'weighted_moving_average_price-BUY-30-sec', 'exp_moving_average_price-BUY-30-sec', 'stochastic_oscillator-BUY-30-sec', 'trade_volume-BUY-36-sec', 'moving_average_price-BUY-36-sec', 'weighted_moving_average_price-BUY-36

In [127]:
X = df[x_columns]
secs = list(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns])))))
delta_sec = secs[1] - secs[0]
max_sec = secs[-1]
min_sec = secs[0]

for column in x_columns:
  name, side, sec, _ = column.split('-')
  if not 'price' in name and int(sec) != min_sec:
    X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']
  elif 'price' in name:
    if int(sec) != max_sec:
      X[f'{name}-{side}-{sec}-sec'] /= X[f'{name}-{side}-{str(int(sec) + delta_sec)}-sec']
    else:
      X = X.drop(f'{name}-{side}-{sec}-sec', axis=1)

X

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,trade_volume-BUY-6-sec,moving_average_price-BUY-6-sec,weighted_moving_average_price-BUY-6-sec,exp_moving_average_price-BUY-6-sec,stochastic_oscillator-BUY-6-sec,trade_volume-BUY-12-sec,moving_average_price-BUY-12-sec,weighted_moving_average_price-BUY-12-sec,exp_moving_average_price-BUY-12-sec,stochastic_oscillator-BUY-12-sec,trade_volume-BUY-18-sec,moving_average_price-BUY-18-sec,weighted_moving_average_price-BUY-18-sec,exp_moving_average_price-BUY-18-sec,stochastic_oscillator-BUY-18-sec,trade_volume-BUY-24-sec,moving_average_price-BUY-24-sec,weighted_moving_average_price-BUY-24-sec,exp_moving_average_price-BUY-24-sec,stochastic_oscillator-BUY-24-sec,trade_volume-BUY-30-sec,moving_average_price-BUY-30-sec,weighted_moving_average_price-BUY-30-sec,exp_moving_average_price-BUY-30-sec,stochastic_oscillator-BUY-30-sec,trade_volume-BUY-36-sec,moving_average_price-BUY-36-sec,weighted_moving_average_price-BUY-36-sec,exp_moving_average_price-BUY-36-sec,stochastic_oscillator-BUY-36-sec,trade_volume-BUY-42-sec,moving_average_price-BUY-42-sec,weighted_moving_average_price-BUY-42-sec,exp_moving_average_price-BUY-42-sec,stochastic_oscillator-BUY-42-sec,trade_volume-BUY-48-sec,moving_average_price-BUY-48-sec,weighted_moving_average_price-BUY-48-sec,exp_moving_average_price-BUY-48-sec,stochastic_oscillator-BUY-48-sec,...,weighted_moving_average_price-SELL-12-sec,exp_moving_average_price-SELL-12-sec,stochastic_oscillator-SELL-12-sec,trade_volume-SELL-18-sec,moving_average_price-SELL-18-sec,weighted_moving_average_price-SELL-18-sec,exp_moving_average_price-SELL-18-sec,stochastic_oscillator-SELL-18-sec,trade_volume-SELL-24-sec,moving_average_price-SELL-24-sec,weighted_moving_average_price-SELL-24-sec,exp_moving_average_price-SELL-24-sec,stochastic_oscillator-SELL-24-sec,trade_volume-SELL-30-sec,moving_average_price-SELL-30-sec,weighted_moving_average_price-SELL-30-sec,exp_moving_average_price-SELL-30-sec,stochastic_oscillator-SELL-30-sec,trade_volume-SELL-36-sec,moving_average_price-SELL-36-sec,weighted_moving_average_price-SELL-36-sec,exp_moving_average_price-SELL-36-sec,stochastic_oscillator-SELL-36-sec,trade_volume-SELL-42-sec,moving_average_price-SELL-42-sec,weighted_moving_average_price-SELL-42-sec,exp_moving_average_price-SELL-42-sec,stochastic_oscillator-SELL-42-sec,trade_volume-SELL-48-sec,moving_average_price-SELL-48-sec,weighted_moving_average_price-SELL-48-sec,exp_moving_average_price-SELL-48-sec,stochastic_oscillator-SELL-48-sec,trade_volume-SELL-54-sec,moving_average_price-SELL-54-sec,weighted_moving_average_price-SELL-54-sec,exp_moving_average_price-SELL-54-sec,stochastic_oscillator-SELL-54-sec,trade_volume-SELL-60-sec,stochastic_oscillator-SELL-60-sec
0,0.100,1.000000,1.000000,1.000000,0.0,0.400,1.000078,1.000113,1.000005,0.000000,10.355,1.000006,1.000000,1.000000,1.000000,0.500,1.000013,1.000001,1.000000,0.000000,10.655,1.000020,1.000002,1.0,1.000000,0.716,1.000023,1.000004,1.000000,0.000000,10.855,1.000019,1.000004,1.000000,1.000000,0.916,1.000000,1.000000,1.0,0.000000,...,1.000067,1.000007,0.000000,0.400,1.000035,1.000038,1.000000,1.000000,0.800,1.000006,1.000005,1.000000,0.000000,0.600,1.000035,1.000140,1.0,1.000000,3.500,1.000020,1.000004,1.000000,0.000000,0.800,1.000023,1.000024,1.0,1.000000,5.566,1.000012,1.000014,1.0,0.000000,3.400,1.000028,1.000018,1.0,1.000000,15.547,0.000000
1,0.200,1.000000,1.000000,1.000000,0.0,0.300,1.000000,1.000000,1.000000,0.000000,0.400,1.000000,1.000000,1.000000,0.000000,0.600,1.000000,1.000000,1.000000,0.000000,0.650,1.000000,1.000000,1.0,0.000000,0.800,1.000037,1.000048,1.000000,0.000000,1.399,1.000005,1.000004,1.000000,0.600000,0.900,0.999985,0.999986,1.0,0.000000,...,1.000000,1.000000,0.000000,0.766,1.000000,1.000000,1.000000,0.000000,4.084,1.000000,1.000000,1.000000,0.000000,0.766,1.000008,1.000003,1.0,0.000000,4.284,1.000025,1.000010,1.000000,0.600000,1.166,1.000000,1.000000,1.0,0.000000,4.284,0.999875,0.999760,1.0,0.600000,34.697,0.999995,1.000000,1.0,-0.339130,4.384,0.600000
2,0.200,1.0

In [260]:
commision = 0.0005
punch = commision * 4

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')
print(y)
print(y.value_counts())

       target
0           0
1           0
2           1
3           0
4           0
...       ...
38742       0
38743       0
38744       0
38745       0
38746       0

[38747 rows x 1 columns]
target
 0        36607
-1         1155
 1          985
dtype: int64


In [261]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)

In [83]:
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        iterations=300,
        eval_metric=metrics.Accuracy(),
        random_seed=126,
        verbose=False,
        loss_function=metrics.MultiClass(),
    )
    
    cv_data = cv(
        Pool(X, y),
        model.get_params(),
        fold_count=4,
        shuffle=True,
        logging_level='Silent',
        stratified=True,
    )

    best_accuracy = np.max(cv_data['test-Accuracy-mean'])
    
    return 1 - best_accuracy


params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=5,
    trials=trials,
    rstate=RandomState(126)
)

print(best)

100%|██████████| 5/5 [29:07<00:00, 349.45s/it, best loss: 0.02841509716654511]
{'l2_leaf_reg': 2.0, 'learning_rate': 0.11805599862470245}


In [189]:
params = {
    'iterations': 300,
    #'l2_leaf_reg': int(best['l2_leaf_reg']),
    #'learning_rate': best['learning_rate'],
    'l2_leaf_reg': 2,
    'learning_rate': 0.1,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
    verbose=False,
)

print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.9841


In [190]:
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[90:100])
print(predictions_probs[90:100])
print(y_test[90:100])

[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[0.24071294 0.53284359 0.22644347]
 [0.00757324 0.98702219 0.00540457]
 [0.00732041 0.97866224 0.01401735]
 [0.00256589 0.99401742 0.00341669]
 [0.00431455 0.9913498  0.00433565]
 [0.00220276 0.99568177 0.00211547]
 [0.02467923 0.93388095 0.04143983]
 [0.00299606 0.99376136 0.00324258]
 [0.00207943 0.99564855 0.00227203]
 [0.00565716 0.98812661 0.00621623]]
       target
20233      -1
18782       0
26657       0
27456       0
36872       0
25891       0
33455       0
23349       0
4176        0
37949       0


In [191]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))


trade_volume-SELL-48-sec: 3.18687421975531
weighted_moving_average_price-BUY-6-sec: 3.1145593713896957
trade_volume-SELL-36-sec: 2.7607401430844862
weighted_moving_average_price-SELL-6-sec: 2.636586787732707
trade_volume-BUY-54-sec: 2.6103906111189032
weighted_moving_average_price-BUY-30-sec: 2.3597990359852705
moving_average_price-SELL-30-sec: 2.3260564853828245
weighted_moving_average_price-BUY-42-sec: 2.3012509879627765
weighted_moving_average_price-BUY-24-sec: 2.2655410616355867
weighted_moving_average_price-SELL-18-sec: 2.166105869538152
trade_volume-SELL-18-sec: 2.0948944335842916
weighted_moving_average_price-BUY-36-sec: 2.0921884928609042
moving_average_price-SELL-6-sec: 2.049479131429462
trade_volume-SELL-12-sec: 1.9668875463982844
moving_average_price-SELL-24-sec: 1.9011850788707298
trade_volume-BUY-36-sec: 1.8940945230158552
trade_volume-SELL-60-sec: 1.8363945183858439
moving_average_price-BUY-12-sec: 1.8343690597159699
weighted_moving_average_price-SELL-48-sec: 1.8245041700

In [206]:
results = []
def f(prob_threshold, diff_threshold, null_threshold):
    p = np.array((list(map(lambda row:
                          1
                          if row[2] > prob_threshold
                              # and row[2] == max(row)
                              and row[2] > row[0]
                              and row[1] < null_threshold
                              and abs(row[2] - row[0]) > diff_threshold
                              and abs(row[2] - row[1]) > diff_threshold
                          else
                          -1
                          if row[0] > prob_threshold
                              # and row[0] == max(row)
                              and row[0] > row[2]
                              and row[1] < null_threshold
                              and abs(row[0] - row[2]) > diff_threshold
                              and abs(row[0] - row[1]) > diff_threshold
                          else 0,
                          predictions_probs))))
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
      t = int(y_test['target'][i:i+1])
      if p[i] == t != 0:
        result['correct_predictions'] += 1
      elif p[i] != t and t != 0 and p[i] != 0:
        result['wrong_side'] += 1
      elif p[i] != 0 and t == 0:
        result['false_positive'] += 1
      elif p[i] == 0 and t != 0:
        result['false_negative'] += 1

    result['correct_predictions'] /= len(y_test)
    result['false_positive'] /= len(y_test)
    result['false_negative'] /= len(y_test)
    result['wrong_side'] /= len(y_test)
    result['prob_threshold'] = prob_threshold
    result['diff_threshold'] = diff_threshold
    result['null_threshold'] = null_threshold
    results.append(result)
    return p

for null_threshold in np.arange(0.5, 0.9, 0.1):
  for prob_threshold in np.arange(0.1, 0.7, 0.1):
      for diff_threshold in np.arange(0.00, 0.20, 0.05):
        f(prob_threshold, diff_threshold, null_threshold)

In [256]:
results = list(sorted(results, key=lambda trade: trade['wrong_side'] - trade['correct_predictions']))

In [257]:
print(results[0])

{'correct_predictions': 0.0018064516129032257, 'false_positive': 0.006580645161290323, 'false_negative': 0.011741935483870968, 'wrong_side': 0.0025806451612903226, 'prob_threshold': 0.1, 'diff_threshold': 0.0, 'null_threshold': 0.8000000000000003}


In [258]:
prob_threshold = results[0]['prob_threshold']
diff_threshold = results[0]['diff_threshold']
null_threshold = results[0]['null_threshold']
print(f"prob_threshold {prob_threshold}, diff_threshold {diff_threshold}, null_threshold {null_threshold}")
p = f(prob_threshold, diff_threshold, null_threshold)
print('accuracy', accuracy_score(p, y_test))
result = {
    'correct_predictions': 0,
    'false_positive': 0,
    'false_negative': 0,
    'wrong_side': 0,
}
for i in range(len(p)):
  t = int(y_test['target'][i:i+1])
  if p[i] == t != 0:
    result['correct_predictions'] += 1
  elif p[i] != t and t != 0 and p[i] != 0:
    result['wrong_side'] += 1
  elif p[i] != 0 and t == 0:
    result['false_positive'] += 1
  elif p[i] == 0 and t != 0:
    result['false_negative'] += 1

result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
result['false_positive_pc'] = result['false_positive'] / len(y_test)
result['false_negative_pc'] = result['false_negative'] / len(y_test)
result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
print(result)

prob_threshold 0.1, diff_threshold 0.0, null_threshold 0.8000000000000003
accuracy 0.9790967741935483
{'correct_predictions': 14, 'false_positive': 51, 'false_negative': 91, 'wrong_side': 20, 'correct_predictions_pc': 0.0018064516129032257, 'false_positive_pc': 0.006580645161290323, 'false_negative_pc': 0.011741935483870968, 'wrong_side_pc': 0.0025806451612903226}


In [259]:
profit = (
      len(y) * result['correct_predictions_pc'] * (punch - 2 * commision)
      -
      len(y) * result['false_positive_pc'] * 2 * commision
      -
      len(y) * result['wrong_side_pc'] * (punch + 2 * commision)
)

print(
    'profit from 3000$ fot 2 months',
    profit * 3000,
    '$',
    profit * 100,
    '%'
)

profit from 3000$ fot 2 months -1544.8803870967743 $ -51.496012903225804 %


In [240]:
correct_accuracy = result['correct_predictions'] / len(y_test[y_test['target'] != 0])
print(
f"Correct accuracy = correct prediction / total positive num = {correct_accuracy}"
)
FPR = result['false_positive'] / len(y_test[y_test['target'] == 0])
print(
f"FPR = false positive / total negative num = {FPR}"
)
FNR = result['false_negative'] / len(y_test[y_test['target'] != 0])
print(
f"FNR = false negative / total positive num = {FNR}"
)
cases_pc = (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) / len(y_test[y_test['target'] != 0]) * 100
probability_pc = result['correct_predictions'] / (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) * 100
print(
f"Model in {cases_pc}% of cases with probability {probability_pc}% predict a correct market jump"
)

Correct accuracy = correct prediction / total positive num = 0.112
FPR = false positive / total negative num = 0.008655737704918032
FNR = false negative / total positive num = 0.704
Model in 82.39999999999999% of cases with probability 13.592233009708737% predict a correct market jump


In [241]:
for i in range(len(predictions_probs)):
  if int(y_test['target'][i:i+1]) != 0:
    print(p[i], predictions_probs[i], int(y_test['target'][i:i+1]))

0 [0.00216579 0.99532284 0.00251136] -1
0 [0.00289282 0.99305316 0.00405403] -1
-1 [0.52756413 0.26394334 0.20849253] 1
-1 [0.24071294 0.53284359 0.22644347] -1
0 [0.00534564 0.98955419 0.00510017] 1
-1 [0.13969526 0.79633907 0.06396568] 1
0 [0.01012872 0.9798413  0.01002998] 1
0 [0.01199594 0.97688973 0.01111434] 1
-1 [0.44669049 0.28253079 0.27077872] 1
0 [0.02510415 0.95534044 0.01955541] 1
0 [0.02387627 0.96043375 0.01568998] 1
-1 [0.68240641 0.07274541 0.24484818] 1
0 [0.00936541 0.9818004  0.00883419] 1
0 [0.00429144 0.99012914 0.00557942] 1
0 [0.03514672 0.92804584 0.03680744] 1
0 [0.00259335 0.99505012 0.00235653] 1
0 [0.00270721 0.99509271 0.00220008] -1
0 [0.03397472 0.94217981 0.02384547] -1
0 [0.00612724 0.97805416 0.0158186 ] 1
0 [0.03335831 0.94909247 0.01754921] -1
0 [0.00157638 0.99655501 0.00186861] 1
0 [0.00992514 0.98209515 0.00797971] -1
0 [0.01486039 0.96072358 0.02441603] 1
0 [0.01047783 0.98075574 0.00876643] -1
0 [0.00887905 0.97905895 0.012062  ] -1
0 [0.039157